# Тестирование модели из MLflow

В этом ноутбуке мы:
1. Загрузим модель из MLflow артефактов
2. Сделаем предсказания на новых данных  
3. Проверим метрики
4. Визуализируем результаты


In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка MLflow
mlflow.set_tracking_uri("file:../mlruns")

print("Библиотеки загружены!")


📦 Библиотеки загружены!


## 1. Поиск последнего эксперимента


In [ ]:
# Получаем информацию об экспериментах
experiment = mlflow.get_experiment_by_name("lesson1-baseline")
if experiment:
    print(f"Найден эксперимент: {experiment.name}")
    print(f"ID: {experiment.experiment_id}")
    
    # Получаем последний run
    runs = mlflow.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["start_time desc"],
        max_results=1
    )
    
    if not runs.empty:
        latest_run = runs.iloc[0]
        run_id = latest_run["run_id"]
        print(f"Последний run: {run_id}")
        print(f"Accuracy: {latest_run['metrics.accuracy']:.4f}")
        print(f"F1-macro: {latest_run['metrics.f1_macro']:.4f}")
    else:
        print("Runs не найдены!")
        run_id = None  # Ensure run_id is defined even if no runs
else:
    print("Эксперимент не найден! Запустите сначала обучение.")
    run_id = None  # Ensure run_id is defined even if no experiment


🎯 Найден эксперимент: lesson1-baseline
📁 ID: 221172159897059064
🚀 Последний run: c3882fbbd6a546128f5f07794c0316cb
📊 Accuracy: 0.9667
📊 F1-macro: 0.9666


## 2. Загрузка модели из MLflow


In [ ]:
# Загружаем модель из артефактов MLflow
if run_id:
    model_uri = f"runs:/{run_id}/model"
    loaded_model = mlflow.sklearn.load_model(model_uri)

    print(f"Модель загружена из: {model_uri}")
    print(f"Тип модели: {type(loaded_model).__name__}")
    print(f"Параметры модели: {loaded_model.get_params()}")
else:
    print("Невозможно загрузить модель: run_id не определен")


✅ Модель загружена из: runs:/c3882fbbd6a546128f5f07794c0316cb/model
🤖 Тип модели: LogisticRegression
🔧 Параметры модели: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 200, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


## 3. Тестирование на данных


In [ ]:
# Загружаем те же данные для тестирования
if run_id and 'loaded_model' in locals():
    iris = load_iris()
    X, y = iris.data, iris.target

    # Используем тот же split что и в обучении (seed=42)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Делаем предсказания
    y_pred = loaded_model.predict(X_test)

    # Вычисляем метрики
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f"Accuracy: {accuracy:.6f}")
    print(f"F1-macro: {f1:.6f}")
    print(f"\\nДетальный отчет:")
    print(classification_report(y_test, y_pred, target_names=iris.target_names))
else:
    print("Невозможно провести тестирование: модель не загружена")


🎯 Accuracy: 0.966667
📊 F1-macro: 0.966583
\n📋 Детальный отчет:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.90      0.95        10
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



## Выводы

**Что мы проверили:**
- Модель успешно загружается из MLflow артефактов
- Результаты полностью воспроизводимы (детерминизм)
- Модель работает на тестовых данных
- Метрики соответствуют ожиданиям

**Преимущества MLflow подхода:**
- Модель хранится с метаданными
- Легко загружать и использовать  
- Версионирование автоматическое
- Воспроизводимые результаты

**Сравнение с простым подходом:**
Попробуйте запустить `basic_train.py` несколько раз и сравните результаты!
